# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# Load the csv exported in Part I to a DataFrame
csv_path = "../output_files/cities_weather.csv"
cities_weather_df = pd.read_csv(csv_path)

cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pevek,69.7008,170.3133,27.52,79,61,14.92,RU,1631359842
1,Hong Kong,22.2855,114.1577,90.97,82,37,1.01,HK,1631359843
2,Mar,43.4002,-4.0094,85.84,72,0,6.91,ES,1631359844
3,Nanortalik,60.1432,-45.2371,44.06,81,100,13.09,GL,1631359845
4,Mochudi,-24.4167,26.1500,75.79,15,0,8.75,BW,1631359845
...,...,...,...,...,...,...,...,...,...
538,Benin,9.5000,2.2500,84.67,67,90,3.09,BJ,1631360338
539,Camocim,-2.9022,-40.8411,85.78,61,55,14.27,BR,1631360339
540,Baturaja,-4.1333,104.1667,81.39,67,96,1.90,ID,1631360339
541,El Puerto de Santa María,36.5939,-6.2330,89.71,58,0,8.05,ES,1631360341


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure the gmaps
gmaps.configure(api_key = g_key)

# Create variable to store Latitude and Longitude
city_lat_lng = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)

# Create figure layout options in dictionary
figure_layout = {"width": "100%",
                 "height": "600px",
                 "padding": "1px"}

# Create heat layer
fig = gmaps.figure(layout = figure_layout, center = (20.0, 10.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_lat_lng, weights = humidity,
                                 dissipating = False, max_intensity = np.max(humidity),
                                 point_radius = 1.5)

# add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Create dataframe for ideal locations with cities meets the requirements:
# A max temperature between 70(F) - 80(F)
# Wind speed less than 10 mph
# Zero cloudiness
vacation_cities_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] >= 70) &
                                           (cities_weather_df["Max Temp"] <= 80) &
                                           (cities_weather_df["Wind Speed"] <= 10) &
                                           (cities_weather_df["Cloudiness"] == 0)].dropna()

vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Mochudi,-24.4167,26.1500,75.79,15,0,8.75,BW,1631359845
24,Pacific,38.4820,-90.7415,71.78,88,0,1.01,US,1631359861
83,Liaocheng,36.4439,115.9647,77.45,83,0,7.81,CN,1631359935
154,Severnyy,50.6770,36.5532,77.04,38,0,2.24,RU,1631359994
166,Sydney,-33.8679,151.2073,70.90,37,0,1.01,AU,1631359601
287,Black,31.0093,-85.7444,70.99,99,0,6.24,US,1631360125
309,Manicoré,-5.8092,-61.3003,75.15,94,0,3.44,BR,1631360144
370,Jinchang,38.4953,102.1739,79.27,33,0,5.84,CN,1631360197


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Drop un-needed columns from the original data frame
hotel_df = vacation_cities_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

# Base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters for the API call
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# Perform API calls to collect hotel name
for index, row in hotel_df.iterrows():
    
    # Insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    # Store into results part of dictionary
    hotel_results = response["results"]
    
    #  Print hotel name and add to data frame if successfully finding hotels
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # Print error statement if 
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")

hotel_df

Mochudi, BW hotel: Penterstar Kgabo Resort, Mochudi
Pacific, US hotel: Quality Inn Near Six Flags St. Louis
Liaocheng, CN hotel: Holiday Inn Express Liaocheng Chiping
Severnyy, RU hotel: Aer Hotel
Sydney, AU hotel: Travelodge Hotel Sydney
No hotel found for this city within 5000 meters.
Manicoré, BR hotel: Hotel Durval
Jinchang, CN hotel: Jinchuan Hotel


,City,Lat,Lng,Country,Hotel Name
4,Mochudi,-24.4167,26.1500,BW,"Penterstar Kgabo Resort, Mochudi"
24,Pacific,38.4820,-90.7415,US,Quality Inn Near Six Flags St. Louis
83,Liaocheng,36.4439,115.9647,CN,Holiday Inn Express Liaocheng Chiping
154,Severnyy,50.6770,36.5532,RU,Aer Hotel
166,Sydney,-33.8679,151.2073,AU,Travelodge Hotel Sydney
287,Black,31.0093,-85.7444,US,
309,Manicoré,-5.8092,-61.3003,BR,Hotel Durval
370,Jinchang,38.4953,102.1739,CN,Jinchuan Hotel


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))